# Day 9: Smoke Basin

[*Advent of Code 2021 day 9*](https://adventofcode.com/2021/day/9) and [*solution megathread*](https://www.reddit.com/rca6vp)

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/UncleCJ/advent-of-code/blob/cj/2021/09/code.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/UncleCJ/advent-of-code/cj?filepath=2021%2F09%2Fcode.ipynb)

In [1]:
from IPython.display import HTML
import sys

sys.path.append('../../')
import common

downloaded = common.refresh()
%store downloaded >downloaded

Writing 'downloaded' (dict) to file 'downloaded'.


## Part One

In [2]:
HTML(downloaded['part1'])

## Boilerplate

Let's try using [pycodestyle_magic](https://github.com/mattijn/pycodestyle_magic) with pycodestyle (flake8 stopped working for me in VS Code Jupyter). Now how does type checking work?

In [3]:
%load_ext pycodestyle_magic

In [4]:
%pycodestyle_on

## Comments

...

In [5]:
testdata = """2199943210
3987894921
9856789892
8767896789
9899965678""".splitlines()

inputdata = downloaded['input'].splitlines()

In [6]:
print(inputdata[:2])

['0197653456789987679545987978921245689901497654212455689767899878999979653246892123569876467899212346', '9989542349899986578959976557890134599892398943201234678956789467989867932135789012398764345678923456']


In [7]:
from collections.abc import Iterable


def parse_data(data: list[str]) -> list[list[int]]:
    return [[int(c) for c in row] for row in data]


def adjacent(heightmap: list[list[int]], x, y) -> list[int]:
    adjacent_coords = []
    if not x == 0:
        adjacent_coords.append((x - 1, y))
    if not x == len(heightmap[0]) - 1:
        adjacent_coords.append((x + 1, y))
    if not y == 0:
        adjacent_coords.append((x, y - 1))
    if not y == len(heightmap) - 1:
        adjacent_coords.append((x, y + 1))
    return [heightmap[y_n][x_n] for x_n, y_n in adjacent_coords]


def low_point(heightmap: list[list[int]], x, y) -> bool:
    return all(heightmap[y][x] < ad for ad in adjacent(heightmap, x, y))


def low_points(heightmap: list[list[int]],
               debug: bool = False) -> list[tuple[int, int, int]]:
    result = []
    for y in range(len(heightmap)):
        for x in range(len(heightmap[0])):
            if low_point(heightmap, x, y):
                if debug:
                    print(f'{x}, {y}: {heightmap[y][x]} is a low point!')
                result.append((x, y, heightmap[y][x]))
            else:
                if debug:
                    print(f'{x}, {y}: {heightmap[y][x]} is not low point!')
    return result


def my_part1_solution(data: str,
                      debug: bool = False) -> int:
    heightmap = parse_data(data)
    return sum(val + 1 for _, _, val in low_points(heightmap, debug))

In [8]:
heightmap = parse_data(testdata)
low_points(heightmap, debug=False)

[(1, 0, 1), (9, 0, 0), (2, 2, 5), (6, 4, 5)]

In [9]:
low_point(heightmap, 9, 0)

True

In [10]:
assert(my_part1_solution(testdata, debug=False) == 15)
# my_part1_solution(testdata, debug=True)

In [11]:
my_part1_solution(inputdata)

585

In [12]:
HTML(downloaded['part1_footer'])

## Part Two

In [13]:
HTML(downloaded['part2'])

In [15]:
import sys;print(sys.version)

1:11: E231 missing whitespace after ';'
1:11: E702 multiple statements on one line (semicolon)


3.9.15 (main, Oct 11 2022, 22:27:25) 
[Clang 14.0.0 (clang-1400.0.29.102)]


In [14]:
from functools import reduce

#   0:      1:      2:      3:      4:
#  aaaa    ....    aaaa    aaaa    ....
# b    c  .    c  .    c  .    c  b    c
# b    c  .    c  .    c  .    c  b    c
#  ....    ....    dddd    dddd    dddd
# e    f  .    f  e    .  .    f  .    f
# e    f  .    f  e    .  .    f  .    f
#  gggg    ....    gggg    gggg    ....

#   5:      6:      7:      8:      9:
#  aaaa    aaaa    aaaa    aaaa    aaaa
# b    .  b    .  .    c  b    c  b    c
# b    .  b    .  .    c  b    c  b    c
#  dddd    dddd    ....    dddd    dddd
# .    f  e    f  .    f  e    f  .    f
# .    f  e    f  .    f  e    f  .    f
#  gggg    gggg    ....    gggg    gggg

# 2 = len(1)
# 3 = len(7)
# 4 = len(4)
# 5 = len(2), len(3), len(5)
# 6 = len(0), len(6), len(9)
# 7 = len(8)

#       a   b   c   d   e   f   g   abs
# 0     1   1   1   0   1   1   1
# 1     0   0   1   0   0   1   0   2
# 2     1   0   1   1   1   0   1
# 3     1   0   1   1   0   1   1
# 4     0   1   1   1   0   1   0   4
# 5     1   1   0   1   0   1   1
# 6     1   1   0   1   1   1   1
# 7     1   0   1   0   0   1   0   3
# 8     1   1   1   1   1   1   1   7
# 9     1   1   1   1   0   1   1
# --
#       8   6   8   7   4   9   7


def number_segments(i: int) -> list[int]:
    match i:
        case 0:
            return [1, 1, 1, 0, 1, 1, 1]
        case 1:
            return [0, 0, 1, 0, 0, 1, 0]
        case 2:
            return [1, 0, 1, 1, 1, 0, 1]
        case 3:
            return [1, 0, 1, 1, 0, 1, 1]
        case 4:
            return [0, 1, 1, 1, 0, 1, 0]
        case 5:
            return [1, 1, 0, 1, 0, 1, 1]
        case 6:
            return [1, 1, 0, 1, 1, 1, 1]
        case 7:
            return [1, 0, 1, 0, 0, 1, 0]
        case 8:
            return [1, 1, 1, 1, 1, 1, 1]
        case 9:
            return [1, 1, 1, 1, 0, 1, 1]
        case _:
            raise ValueError(f'Not an integer between 0-9: {i}')


def numbers_segments() -> dict[int, list[int]]:
    return {i: number_segments(i) for i in range(10)}


def elem_add(a: list, b: list) -> list:
    assert(len(a) == len(b))
    return [a_i + b_i for a_i, b_i in zip(a, b)]


def elem_subtract(a: list, b: list) -> list:
    assert(len(a) == len(b))
    return [a_i - b_i for a_i, b_i in zip(a, b)]


def elems_add(ls: Iterable[list]) -> list:
    return reduce(elem_add, ls)


# def pattern_list(pattern: str) -> list[int]:
#     return [1 if for i, s in enumerate('abcdefg'):


def sort_wires(testpatterns) -> dict[str, str]:
    tests_sum = elems_add()


def my_part2_solution(data: str,
                      debug: bool = False) -> int:
    numbers_s = numbers_segments()
    numbers_sum = elems_add(numbers_s.values())

    for testpatterns, outputpatterns in parse_data(data):
        print(f'{testpatterns=} {outputpatterns=}')

    return 0

SyntaxError: invalid syntax (3925735858.py, line 44)

Error before execution: invalid syntax (3925735858.py, line 44)


In [ ]:
my_part2_solution(testdata)

In [ ]:
# assert(my_part2_solution(testdata) == 168)

In [ ]:
# my_part2_solution(inputdata)

In [ ]:
HTML(downloaded['part2_footer'])